In [ ]:
import pandas as pd

In [ ]:
wecc_regions = pd.read_csv('wecc_counties_population.csv', index_col=0)

# state level profiles from EFS

In [ ]:
df = pd.read_csv('/Users/siobhanpowell/Downloads/EFSLoadProfile_Reference_Moderate.csv')
efs_loads = pd.DataFrame({'Hour':np.arange(1, 8761)})
for state in ['AZ', 'CA', 'CO', 'ID', 'MT', 'NE', 'NM', 'ND', 'NV', 'OK', 'OR', 'SD', 'TX', 'UT', 'WA', 'WY']:
    for year in [2018, 2020, 2024, 2030, 2040, 2050]:
        local_load = df.loc[(df['Year']==year)&(df['State']==state)&(~(df['Subsector']=='light-duty vehicles'))].groupby(by='LocalHourID').sum()['LoadMW']
        efs_loads[state+'_'+str(year)+'_nonLDV'] = local_load.values
        local_load = df.loc[(df['Year']==year)&(df['State']==state)&((df['Subsector']=='light-duty vehicles'))].groupby(by='LocalHourID').sum()['LoadMW']
        efs_loads[state+'_'+str(year)+'_LDV'] = local_load.values

# convert from state level to wecc level

In [ ]:
states = np.sort(list(set(wecc_regions['State_name'])))
state_shortforms = {'Arizona':'AZ', 'California':'CA', 'Colorado':'CO', 'Idaho':'ID', 'Montana':'MT', 
                    'Nebraska':'NE', 'Nevada':'NV', 'New Mexico':'NM', 'North Dakota':'ND', 'Oklahoma':'OK', 
                    'Oregon':'OR', 'South Dakota':'SD', 'Texas':'TX', 'Utah':'UT', 'Washington':'WA', 'Wyoming':'WY'}
for year in [2018, 2020, 2024, 2030, 2040, 2050]:
    efs_loads['WECC_'+str(year)+'_LDV'] = 0
    efs_loads['WECC_'+str(year)+'_nonLDV'] = 0
    for state in states:
        frac = wecc_regions.loc[(wecc_regions['State_name']==state)]['Population'].sum() / regions1.loc[regions1['State_name']==state]['Population'].sum()
        efs_loads['WECC_'+str(year)+'_LDV'] += frac*efs_loads[state_shortforms[state]+'_'+str(year)+'_LDV'].values
        efs_loads['WECC_'+str(year)+'_nonLDV'] += frac*efs_loads[state_shortforms[state]+'_'+str(year)+'_nonLDV'].values
          
# efs_loads.to_csv('electrification_futures_baseline.csv')

In [ ]:
# efs_loads = pd.read_csv('electrification_futures_baseline.csv', index_col=0)

# extract scaling factor: 

In [ ]:
print((efs_loads['WECC_2030_nonLDV'] / efs_loads['WECC_2018_nonLDV']).mean())
print((efs_loads['WECC_2030_nonLDV'] / efs_loads['WECC_2018_nonLDV']).median())